<a href="https://colab.research.google.com/github/jason-zl190/deeplearning/blob/master/test_vggLoss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, optimizers, metrics
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np

#Build Simple Model
inputs = keras.Input(shape=(32,32,3))
x = Conv2D(16, 3, padding = 'same', activation='relu')(inputs)
x = Conv2D(16, 3, padding = 'same', activation='relu')(x)
o = Conv2D( 3, 3, padding = 'same', activation='relu')(x)
model = keras.Model(inputs=inputs, outputs=o)

#Generate dummy data
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.
y_train = x_train.copy()
x_train = x_train+0.1*np.random.randn(*x_train.shape).astype('f')
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(128)

#Load VGG model
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape = [32,32,3])
vgg.trainable = False
content_layers = 'block5_conv2'

lossModel = models.Model([vgg.input], vgg.get_layer(content_layers).output, name = 'vggL')

def lossVGG(X,Y):
    Xt = preprocess_input(X*255)
    Yt = preprocess_input(Y*255)
    vggX = lossModel(Xt)
    vggY = lossModel(Yt)
    return tf.reduce_mean(tf.square(vggY-vggX))

optimizer = optimizers.Adam(learning_rate=0.0005)
loss = [lossVGG]
model.compile(optimizer = optimizer, loss = loss)
history = model.fit(train_ds, epochs = 20)